In [1]:
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import torch
import torchvision
import os
import glob

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [4]:
from torchvision import models
import torch
import torch.nn as nn
import torch.optim as optim

resnet50 = models.resnet50(pretrained=False).to(device)

C:\Users\Administrator\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Administrator\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [5]:
resnet50.fc = nn.Linear(resnet50.fc.in_features, 3).to(device)

In [6]:
from torchsummary import summary

state_dict_path='C:/Users/Administrator/DCD/best_model_weight3.pt'
weights = torch.load(state_dict_path, map_location = 'cpu')
resnet50.load_state_dict(weights['net'])
# summary(resnet50, input_size=(3, 224, 224))
resnet50.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [7]:
import os
 
def createFolder(directory, folder):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
            return folder
        else:
            folder = int(os.path.basename('C:/Users/Administrator/Webcam/{}'.format(folder)))
            folder += 1
            return createFolder('C:/Users/Administrator/Webcam/{}'.format(folder), folder)
            
    except OSError:
        print ('Error: Creating directory. ' +  directory)


In [8]:
def img_modify(img):
    # 캡쳐한 이미지 변환
    img = Image.fromarray(frame)     # array -> PIL
    img_resize = img.resize((224, 224)) # 244x244 resize
    x = np.array(img_resize) # PIL -> array

    tf_toTensor = ToTensor()
    tensor_img = tf_toTensor(x) # array -> tensor
    tensor_img = tensor_img.unsqueeze(0) # tensor demension 추가
    return tensor_img

In [9]:
def prediction(img):
        output = resnet50(img)
        pred = torch.argmax(output, 1) # 가장 높은 확률로 예측한 index 출력
        
        return int(pred)

In [10]:
def flood_fense(pred, cnt):
    
    if pred == 0:             # 0 = flood
        print('  현재상태  : 침수')
        cnt += 1
    elif pred == 1:             # 1 = not_flood
        print('  현재상태  : 이상없음')
        cnt -= 3
    else:                          # 2 = rain
        print('  현재상태  : 비')
        cnt -= 3
        
    if cnt < 0:
        cnt = 0
    if cnt > 0:
        print("Flood Count : ", cnt)
    
    return cnt

In [11]:
def act_alert():
    print("\n침수 경보\n")

In [12]:
def act_fence(cnt):
    print("\n[차수막 가동]")
    cnt = -1
    return cnt

In [13]:
def turn_off(cnt):
    cnt = 0
    print("[차수막 작동 중지]\n")
    return cnt

In [14]:
import requests
from bs4 import BeautifulSoup


url = 'https://www.weather.go.kr/w/obs-climate/land/city-obs.do'
response = requests.get(url)

def rainfall(city):

    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        for i in range(1,95):
            title = soup.select_one('#weather_table > tbody > tr:nth-child({}) > td:nth-child(1) > a'.format(i))

            if city == title.get_text():
                precipitation = soup.select_one('#weather_table > tbody > tr:nth-child({}) > td:nth-child(9)'.format(i))
                precipitation = precipitation.get_text()
                if precipitation == "\xa0":
                    precipitation = 0
                print(title.get_text()+" 강수량 : ", precipitation)
            else:
                precipitation = 50
    else :
        print(response.status_code)
    return precipitation

In [15]:
def rain_detect(pred, precipitation):
    int(precipitation)
    if precipitation <= 30:
        return 1
    elif 30 < precipitation:
        return pred

In [ ]:
import cv2
from PIL import Image
import numpy as np
from torchvision.transforms import ToTensor, ToPILImage
# import serial
# import time

# ser = serial.Serial('COM3', 9600)

city = input("현재 지역 : ")

webcam = cv2.VideoCapture(0)

folder_num = 0
time_num= 0    
image_num = 0
count = 0

folder_num = createFolder('C:/Users/Administrator/Webcam/{}'.format(folder_num), folder_num)

#-- 웹캠 열기
while webcam.isOpened():
    
    if time_num%54000 == 0.0:
        precipitation = rainfall(city)
        time_num = 1

    status, frame = webcam.read()
    time_num = time_num + 1
    
    if not status:
        break

    #-- 출력창
    cv2.imshow("WebCam", frame)
    
    #-- 캡쳐 프레임 간격 설정
    if time_num%30 == 0.0:
        image_num = image_num + 1
        cv2.imwrite('{}/screenshot{}.png'.format(folder_num, image_num), frame, params=[cv2.IMWRITE_PNG_COMPRESSION,0])
        
        image = img_modify(frame) # 이미지 변환

        pred = prediction(image) # 예측하기
        
        pred = rain_detect(pred, precipitation) # 비가 오는지 판단
        
        # Flood Count 계산
        # 차수막의 오작동을 방지하기 위해 Flood Count 적용
        if count != -1:                     # count 가 -1 이면 차수막 가동중
            count = flood_fense(pred, count)
            if 15 <= count < 30:
                act_alert()                 # count가 15이상이면 경보
            elif count == 30:
#                 val = 1
#                 val = val.encode('utf-8')
#                 ser.write(val)
                count = act_fence(count)   # count가 30이상이면 차수막 가동
        else:
            print("차수막 가동중")
    
    #-- 0 입력시 종료 
    if cv2.waitKey(1) & 0xFF == ord('0'):
        break
    elif cv2.waitKey(1) & 0xFF == ord('1'): # 차수막 내리기
        if count == -1:
            count = turn_off(count)         # count를 0으로 초기화
            
webcam.release()
cv2.destroyAllWindows()

현재 지역 : 
  현재상태  : 이상없음
  현재상태  : 이상없음
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 침수
Flood Count :  1
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 이상없음
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 이상없음
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 침수
Flood Count :  1
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 이상없음
  현재상태  : 비
  현재상태  : 이상없음
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
  현재상태  : 비
 

In [14]:
from bs4 import BeautifulSoup


bs4.BeautifulSoup